<a href="https://colab.research.google.com/github/Malavika-Gupta/Smart-Vision-Technology-Quality-Control-Flipkart-GRID-6.0-/blob/main/Freshness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing required libraries.

import numpy as np
import pandas as pd
import os
import cv2

In [2]:
!pip install opendatasets
!pip install pandas

In [3]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/muhriddinmuxiddinov/fruits-and-vegetables-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: lukeverlac
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/muhriddinmuxiddinov/fruits-and-vegetables-dataset


100%|██████████| 870M/870M [00:41<00:00, 22.1MB/s]


In [4]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.optim as optim
from sklearn.metrics import accuracy_score


In [5]:
print(os.listdir('/content'))

['.config', 'fruits-and-vegetables-dataset', 'sample_data']


In [6]:
print(os.listdir('/content/fruits-and-vegetables-dataset'))

['Fruits_Vegetables_Dataset(12000)']


In [7]:
print(os.listdir('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)'))


['Vegetables', 'Fruits']


In [8]:
# List contents of the Vegetables folder
print(os.listdir('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Vegetables'))

# List contents of the Fruits folder
print(os.listdir('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Fruits'))


['FreshPotato', 'FreshBellpepper', 'FreshTomato', 'RottenBellpepper', 'RottenCucumber', 'RottenTomato', 'RottenPotato', 'FreshCucumber', 'FreshCarrot', 'RottenCarrot']
['RottenApple', 'FreshStrawberry', 'RottenMango', 'RottenBanana', 'FreshBanana', 'FreshApple', 'RottenOrange', 'FreshOrange', 'RottenStrawberry', 'FreshMango']


In [9]:
from sklearn.model_selection import train_test_split

# Define the base directory for Vegetables and Fruits
base_dir = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)'

# Dictionary to map folder names to labels (1 for fresh, 0 for rotten)
label_mapping = {
    'Fresh': 1,
    'Rotten': 0
}

# Subfolders for vegetables and fruits
vegetable_folders = ['FreshTomato', 'FreshCucumber', 'RottenPotato', 'FreshBellpepper',
                     'RottenBellpepper', 'RottenCucumber', 'FreshCarrot', 'RottenTomato',
                     'FreshPotato', 'RottenCarrot']

fruit_folders = ['RottenStrawberry', 'FreshApple', 'RottenMango', 'FreshStrawberry',
                 'FreshMango', 'RottenOrange', 'FreshBanana', 'RottenBanana',
                 'RottenApple', 'FreshOrange']

# Function to load and preprocess images from each folder
def load_images_from_folder(base_dir, folders, image_size=(100, 100)):
    images = []
    labels = []

    for folder in folders:
        # Determine if it's fresh or rotten based on the folder name
        label = label_mapping['Fresh'] if 'Fresh' in folder else label_mapping['Rotten']
        folder_path = os.path.join(base_dir, folder)

        # Iterate through each image in the folder
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize image to a consistent size (100x100 pixels)
                img = cv2.resize(img, image_size)
                images.append(img)
                labels.append(label)

    return images, labels

# Load vegetable images
vegetable_dir = os.path.join(base_dir, 'Vegetables')
vegetable_images, vegetable_labels = load_images_from_folder(vegetable_dir, vegetable_folders)

# Load fruit images
fruit_dir = os.path.join(base_dir, 'Fruits')
fruit_images, fruit_labels = load_images_from_folder(fruit_dir, fruit_folders)

# Combine images and labels for both fruits and vegetables
X = np.array(vegetable_images + fruit_images)
y = np.array(vegetable_labels + fruit_labels)

# Normalize the image data (scale pixel values between 0 and 1)
X = X / 255.0

# Split dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print dataset sizes
print(f'Training data: {X_train.shape}, Test data: {X_test.shape}')

Training data: (9600, 100, 100, 3), Test data: (2400, 100, 100, 3)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout


# Build a CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification (fresh vs rotten)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.6106 - loss: 0.6446 - val_accuracy: 0.7738 - val_loss: 0.4797
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7956 - loss: 0.4565 - val_accuracy: 0.8225 - val_loss: 0.3721
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8388 - loss: 0.3768 - val_accuracy: 0.8529 - val_loss: 0.3471
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8638 - loss: 0.3112 - val_accuracy: 0.8546 - val_loss: 0.3346
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8883 - loss: 0.2780 - val_accuracy: 0.8121 - val_loss: 0.3871
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9037 - loss: 0.2446 - val_accuracy: 0.8971 - val_loss: 0.2551
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9216 - loss: 0.1981 - val_accuracy: 0.8846 - val_loss: 0.2833
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9200 - loss: 0.1981 - val_acc

In [16]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9147 - loss: 0.2577
Test Accuracy: 90.83%
